In [17]:
import pandas as pd
import glob
import numpy as np
from fuzzywuzzy import fuzz
from openpyxl import load_workbook

In [18]:
immport = pd.read_excel("/Users/liualg/Documents/GitHub/SeroNet/seronetTemplates/template/experimentSamples.Other.xlsx",
              sheet_name="lookup",
              header=None
        
            )[0].values.tolist()

In [19]:
seronet = pd.read_excel(
    glob.glob("/Users/liualg/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Curation channel/*.xlsm")[0],
                        sheet_name=1,
                        header=None).to_numpy()

json_seronet_dict = pd.read_excel("./dictionary/Facetdict.xlsx",
                                 header=None)
json_seronet_dict = dict(
    zip(json_seronet_dict.loc[:][0],json_seronet_dict.loc[:][1])
)

In [20]:
# creating a dictionary with each look up table
wb = load_workbook(
    glob.glob("/Users/liualg/Library/CloudStorage/" \
              "OneDrive-NationalInstitutesofHealth/Curation channel/*.xlsm")[0],
                  )
sheet = wb['Cntrl\'d Vocab']
immport_dict = dict()

for column in sheet.iter_cols( #sheet.max_column, 
                              max_row=sheet.max_row, 
                              min_row=4):
#     try:
    name = []
    start_index = []
    for icol, cell in enumerate(column):
        if icol == 0:
            name.append(cell.value)
            start_index.append(icol)
        else:
            if (cell.value is not None and column[icol-1].value is None):
                name.append(cell.value)
                start_index.append(icol)
    # FIGURE THIS SHIT OUT
    try:
        for i in range(len(start_index)-1):
            immport_dict[name[i]] = [k.value for k in column[start_index[i] : start_index[i+1]] if k.value is not None]
        
        immport_dict[name[-1]] = [k.value for k in column[start_index[i-1] :] if k.value is not None]
    
    except:
        immport_dict[name[-1]] = [k.value for k in column[start_index[i] :] if k.value is not None]
        pass
    
    
#     except:
#         print(column[0])
    

               

In [21]:
json_seronet_dict.get('assay_type')

'Assay Type'

In [22]:
def get_closest_lookup(word, lookup_table):
    correct_word = ''
    closest_lookup = ''
    check_list = []
    
    if word in lookup_table:
        closest_lookup = word

    else:
        for potential in lookup_table:
            if fuzz.ratio(word, potential) > 70: ##use math for this
                check_list.append((potential, 
                                  fuzz.ratio(word, potential))
                                 )
            else: 
                pass

            # this is a little weird 
        if len(check_list) == 1:
            print(f"[FYI] fuzzywuzzy:: {word} => {check_list[0][0]} : score {check_list[0][1]}")
            closest_lookup = check_list[0][0]

        elif len(check_list) > 1:
            check_list.sort(key=lambda a: a[0])
            print(f'Top three choices:')
            print(check_list[:3],sep='\n')
            closest_lookup = check_list[int(input('Designate which choice to use: '))-1][0]

        else:
            print(f"ERROR:: Please check spelling for: {word}")
            
    return closest_lookup
            

        


In [23]:
def get_closest_lookup(word, lookup_table):
    correct_word = ''
    closest_lookup = ''
    check_list = []
    
    if word in lookup_table:
        closest_lookup = word

    else:
        for potential in lookup_table:
            if fuzz.ratio(word, potential) >= 70: ##use math for this
                check_list.append((potential, 
                                  fuzz.ratio(word, potential))
                                 )

            elif 50 <= fuzz.ratio(word, potential) < 70: ##use math for this
                check_list.append((potential, 
                                  fuzz.ratio(word, potential))
                                 )
            else: 
                pass

            # this is a little weird 
        if len(check_list) == 1:
            print(f"[FYI] fuzzywuzzy:: {word} => {check_list[0][0]} : score {check_list[0][1]}")
            closest_lookup = check_list[0][0]

        elif len(check_list) > 1:
            check_list.sort(key=lambda a: a[0])
            print(f'\n\n######  ACTION REQUIRED ###### \nTop three choices:')
            print(check_list[:3])
            closest_lookup = check_list[int(input('Designate which choice to use: '))-1][0]

        else:
            print(f"ERROR:: Please check spelling for: {word}")

    return closest_lookup

def check_spelling(word_in_questions, facet_to_check):
    lk_table = immport_dict.get(json_seronet_dict.get(facet_to_check))

    if isinstance(word_in_questions,str):
        correct_word = get_closest_lookup(word_in_questions,lk_table)

    else:
        correct_word = []
        for words in word_in_questions:
            correct_word.append(get_closest_lookup(words,lk_table))

    return correct_word

In [24]:
Covid-19 | SARS-CoV-2 Alpha; B.1.1.7 | SARS-CoV-2 Delta; B.1.617.2

SyntaxError: invalid syntax (243581567.py, line 1)

In [28]:
check_spelling('Covid-19','reported_health_condition')

'Covid-19'

In [27]:
'Covid-19' in immport_dict.get(json_seronet_dict.get('reported_health_condition'))

True

In [48]:
check_list = [('AstraZeneca SARS-CoV-2 vaccine', 18), ('CureVac AG SARS-CoV-2 vaccine', 18), ('Johnson & Johnson SARS-CoV-2 vaccine', 12), ('Moderna SARS-CoV-2 vaccine', 24), ('N/A', 11), ('Other SARS-CoV-2 vaccine', 26), ('Pfizer-BioNTech SARS-CoV-2 vaccine', 61), ('SARS-CoV-2 Vaccine Type - STUDY', 17), ('Sinopharm SARS-CoV-2 vaccine', 23), ('Sinovac SARS-CoV-2 vaccine', 20), ('Sputnik V SARS-CoV-2 vaccine', 19)]

In [49]:
check_list.sort(key=lambda a: a[0])

In [51]:
print(check_list)

[('AstraZeneca SARS-CoV-2 vaccine', 18), ('CureVac AG SARS-CoV-2 vaccine', 18), ('Johnson & Johnson SARS-CoV-2 vaccine', 12), ('Moderna SARS-CoV-2 vaccine', 24), ('N/A', 11), ('Other SARS-CoV-2 vaccine', 26), ('Pfizer-BioNTech SARS-CoV-2 vaccine', 61), ('SARS-CoV-2 Vaccine Type - STUDY', 17), ('Sinopharm SARS-CoV-2 vaccine', 23), ('Sinovac SARS-CoV-2 vaccine', 20), ('Sputnik V SARS-CoV-2 vaccine', 19)]


In [53]:
c = [('Asasdad', 99),
 ('AstraZeneca SARS-CoV-2 vaccine', 54),
 ('AstraZenecazxcdfaa', 23),
 ('CureVac AG SARS-CoV-2 vaccine', 20),
 ('Johnson & Johnson SARS-CoV-2 vaccine', 17),
 ('sdds', 1)]

In [74]:
c.sort(key=lambda a: a[1], reverse=True)

In [77]:
print(c[:3])

[('Asasdad', 99), ('AstraZeneca SARS-CoV-2 vaccine', 54), ('AstraZenecazxcdfaa', 23)]


In [57]:
type(check_list)

list

In [60]:
check_list[0][1]

18